In [ ]:
install.packages("readr")
library(readr)

In [ ]:
install.packages("dplyr")
library(dplyr)

Importar la Data

In [ ]:
data <- read.csv("C:/Users/johnb/Documents/GitHub/causalai-2025/R/input/apartments.csv")

In [ ]:
names(data)

In [ ]:
head(data)

3a : data cleaning

In [ ]:
area2=data$area^2

In [ ]:
data$area2<-area2

In [ ]:
data <- data %>%
    mutate(
        hasparkingspace = recode(hasparkingspace, "yes" = 1, "no" = 0),
        hasbalcony = recode(hasbalcony, "yes" = 1, "no" = 0),
        haselevator = recode(haselevator, "yes" = 1, "no" = 0),
        hassecurity = recode(hassecurity, "yes" = 1, "no" = 0),
        hasstorageroom = recode(hasstorageroom, "yes" = 1, "no" = 0)
    )
    


In [ ]:
ud <- data$area %% 10

In [ ]:
data$ud<-ud

In [ ]:
for (i in 0:9){
    data[[paste0("ud_",i)]]<-ifelse(data$ud==i,1,0)
    }

3b : linear model estimation

In [ ]:
data$month<-as.factor(data$month)
data$type<-as.factor(data$type)
data$rooms<-as.factor(data$rooms)
data$ownership<-as.factor(data$ownership)
data$buildingmaterial<-as.factor(data$buildingmaterial)

In [ ]:
model<-lm(price~ud_0+ud_1+ud_2+ud_3+ud_4+ud_5+ud_6+ud_7+ud_8+area+area2+schooldistance+clinicdistance+postofficedistance+
          kindergartendistance+collegedistance+pharmacydistance+ownership+buildingmaterial+hasparkingspace+hasbalcony+haselevator+
          hassecurity+hasstorageroom, data=data)

In [ ]:
tabla<-summary(model)

In [ ]:
print(tabla)

**Comentario**:/n
Lo que se ve en el coeficiente de ud_0 es un valor de 1.717e+04 y con una significancia de p-val 4.97e-05 ***./n
Esto indica que existe una diferencia significativa entre la categoria base (area cuyo ulitmo digito termina es 9) y areas de departamentos /n
que finalizan en 0. En concreto las areas con un metraje de ud0 tienen un precio 17170 polish zlotes mas alto en promedio que un departamento que no termina en 9.

**partialing out**

In [ ]:
residuos_mc<-model$residuals

In [ ]:
modelsinud0<-lm(price~ud_1+ud_2+ud_3+ud_4+ud_5+ud_6+ud_7+ud_8+area+area2+schooldistance+clinicdistance+postofficedistance+
          kindergartendistance+collegedistance+pharmacydistance+ownership+buildingmaterial+hasparkingspace+hasbalcony+haselevator+
          hassecurity+hasstorageroom, data=data)

In [ ]:
residuos_msinud0<-modelsinud0$residuals

In [ ]:
modelud0<-lm(ud_0~ud_1+ud_2+ud_3+ud_4+ud_5+ud_6+ud_7+ud_8+area+area2+schooldistance+clinicdistance+postofficedistance+
          kindergartendistance+collegedistance+pharmacydistance+ownership+buildingmaterial+hasparkingspace+hasbalcony+haselevator+
          hassecurity+hasstorageroom, data=data)

In [ ]:
residuosud0<-modelud0$residuals

In [ ]:
p0model<-lm(residuos_msinud0~residuosud0,data=data)

In [ ]:
summary(p0model)

In [ ]:
coef(model)[2]

In [ ]:
coef(p0model)[2]

Comentario: *Como se puede ver el modelo con residuos y el modelo completo producen el mismo resultado*

3c: price premium for area that ends in 0 digit

3c.1 data train

In [ ]:
datasinud0<-filter(data,ud_0==0)

In [ ]:
modelnoud0<-lm(price~ud_1+ud_2+ud_3+ud_4+ud_5+ud_6+ud_7+ud_8+area+area2+schooldistance+clinicdistance+postofficedistance+
          kindergartendistance+collegedistance+pharmacydistance+ownership+buildingmaterial+hasparkingspace+hasbalcony+haselevator+
          hassecurity+hasstorageroom, data=datasinud0)

predecir model con los coeficientes con modelnoud0


predecir modelo con data sin ud0

In [ ]:
price_predicted<-predict(modelnoud0,data)

In [ ]:
data.frame(price_predicted,data$ud_0)

In [ ]:
ver<-data.frame(data$price, price_predicted)

In [ ]:
ver

3c.3 : comparar promedios

In [ ]:
data$predecido<-price_predicted

In [ ]:
dataud0<-filter(data,ud_0==1)

In [ ]:
avgprice<-mean(dataud0$price)

In [ ]:
 avgpredicted<-mean(dataud0$predecido,na.rm=T)

In [ ]:
Comparisson=data.frame(avgprice,avgpredicted)

In [ ]:
Comparisson

**Comentario** : el sesgo viene por parte de que las personas se fijan mas en que el terreno termine en 0 , que en las caracteristicas propias del departamento como numero de cuartos o si tiene asensor. 